<a href="https://colab.research.google.com/github/Ragavi203/Smart-Contract-Audit-Assistant/blob/main/smart_contract_audit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#Smart Contract Audit Assistant

!pip install anthropic py-solc-x

import os
import json
import anthropic
from solcx import compile_standard, install_solc

# STEP 1: Install Solidity Compiler
install_solc("0.8.20")

# STEP 2: Paste your Solidity contract and imported contracts
solidity_code = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC20/IERC20.sol";

contract MyContract {
    IERC20 public token;
    mapping(address => uint256) public balances;

    constructor(address _token) {
        token = IERC20(_token);
    }

    function deposit(uint256 amount) public {
        require(amount > 0, "Amount must be greater than 0");
        token.transferFrom(msg.sender, address(this), amount);
        balances[msg.sender] += amount;
    }

    function withdraw(uint256 amount) public {
        require(amount <= balances[msg.sender], "Insufficient balance");
        balances[msg.sender] -= amount;
        token.transfer(msg.sender, amount);
    }

    function emergencyWithdraw() public {
        uint256 amount = balances[msg.sender];
        balances[msg.sender] = 0;
        token.transfer(msg.sender, amount);
    }
}
'''

# Content of the imported IERC20.sol contract (simplified for demonstration)
# In a real scenario, you might need the full contract or a more specific version.
ierc20_code = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

interface IERC20 {
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);

    function totalSupply() external view returns (uint256);
    function balanceOf(address account) external view returns (uint256);
    function transfer(address to, uint256 amount) external returns (bool);
    function allowance(address owner, address spender) external view returns (uint256);
    function approve(address spender, uint256 amount) external returns (bool);
    function transferFrom(address from, address to, uint256 amount) external returns (bool);
}
'''


# STEP 3: Compile and extract AST
compiled = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            "MyContract.sol": {"content": solidity_code},
            "@openzeppelin/contracts/token/ERC20/IERC20.sol": {"content": ierc20_code} # Include the imported contract
            },
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "evm.bytecode"],
                    "": ["ast"]
                }
            }
        }
    },
    solc_version="0.8.20"
)
ast = compiled['sources']['MyContract.sol']['ast']

# STEP 4: Pattern Matching on AST
def detect_issues(ast):
    warnings = []

    def traverse(node):
        if isinstance(node, dict):
            if node.get('nodeType') == 'FunctionCall':
                if node.get('expression', {}).get('memberName') == 'transfer':
                    # Check if the transfer is on an address type (ETH transfer) or token (IERC20 transfer)
                    # This is a simplified check and might need refinement for complex cases.
                    try:
                        # If the transfer is on a MemberAccess and the baseExpression is an Identifier
                        # with name 'token', assume it's an ERC20 transfer.
                        if not (node.get('expression', {}).get('nodeType') == 'MemberAccess' and
                                node.get('expression', {}).get('baseExpression', {}).get('nodeType') == 'Identifier' and
                                node.get('expression', {}).get('baseExpression', {}).get('name') == 'token'):
                             warnings.append("Detected direct ETH `.transfer()` usage — may be unsafe due to fixed gas limit.")

                    except:
                         warnings.append("Detected direct ETH `.transfer()` usage — may be unsafe due to fixed gas limit.")


            if node.get('nodeType') == 'FunctionDefinition':
                body = node.get('body', {}).get('statements', [])
                state_updated = False
                for stmt in body:
                    # A more robust check for state updates might be needed
                    if 'Assignment' in str(stmt) or 'Delete' in str(stmt):
                         state_updated = True

                    # Check for external calls before state updates
                    if node.get('name') != 'constructor': # Exclude constructor from reentrancy check
                        if 'transfer' in str(stmt) or 'call' in str(stmt) or 'send' in str(stmt):
                            if not state_updated:
                                warnings.append(f"Possible reentrancy risk in `{node['name']}`: external call before state update.")

            for k, v in node.items():
                traverse(v)
        elif isinstance(node, list):
            for child in node:
                traverse(child)

    traverse(ast)
    return warnings

issues = detect_issues(ast)

# STEP 5: Claude API Call
# Ensure you have set your ANTHROPIC_API_KEY in the environment variables
# os.environ["ANTHROPIC_API_KEY"] = "YOUR_CLAUDE_API_KEY" # Uncomment and replace if not using environment variable

if "ANTHROPIC_API_KEY" not in os.environ:
    print("Please set the ANTHROPIC_API_KEY environment variable.")
    llm_feedback = "Claude API key not set."
else:
    client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

    def audit_with_claude(code):
        prompt = f"""
    You're a Solidity smart contract auditor.

    Analyze the code and provide:
    1. Security issues
    2. Gas inefficiencies
    3. Best practices

Code:
{code}
"""

        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1000,
            temperature=0.2,
            system="You are a Solidity smart contract auditor.",
            messages=[{"role": "user", "content": prompt}]
        )

        return response.content[0].text

    llm_feedback = audit_with_claude(solidity_code)

# STEP 6: Generate Final Report
from IPython.display import Markdown

def generate_final_report(code, static_issues, llm_feedback):
    md = "# 🧾 Smart Contract Audit Report\n\n"
    md += "## 🔍 Static AST Analysis\n"
    if static_issues:
        for issue in static_issues:
            md += f"- {issue}\n"
    else:
        md += "No known static issues found.\n"

    md += "\n---\n\n## 🤖 Claude Audit Result\n"
    md += llm_feedback
    return md

Markdown(generate_final_report(solidity_code, issues, llm_feedback))

# 🧾 Smart Contract Audit Report

## 🔍 Static AST Analysis
- Possible reentrancy risk in `deposit`: external call before state update.
- Possible reentrancy risk in `withdraw`: external call before state update.
- Detected direct ETH `.transfer()` usage — may be unsafe due to fixed gas limit.
- Possible reentrancy risk in `emergencyWithdraw`: external call before state update.
- Detected direct ETH `.transfer()` usage — may be unsafe due to fixed gas limit.

---

## 🤖 Claude Audit Result
Thank you for providing the Solidity smart contract code. Here's my analysis:

1. **Security Issues:**
   - **Reentrancy Attack**: The contract is vulnerable to a reentrancy attack. The `withdraw` function updates the user's balance before transferring the tokens, which could allow an attacker to repeatedly call the `withdraw` function before the balance is updated, draining the contract's funds. To mitigate this, you should use the "Checks-Effects-Interactions" pattern and update the user's balance before the token transfer.
   - **Lack of Access Control**: The contract does not have any access control mechanisms, which means anyone can call the `deposit`, `withdraw`, and `emergencyWithdraw` functions. This could lead to unauthorized access and potential misuse of the contract. You should consider implementing access control, such as using the `onlyOwner` modifier or a role-based access control system.

2. **Gas Inefficiencies:**
   - **Unnecessary Storage Access**: The `balances` mapping is updated in both the `deposit` and `withdraw` functions. This can lead to increased gas costs, as each storage access operation is relatively expensive. Consider optimizing the contract by reducing the number of storage accesses, for example, by using a single function to handle both deposits and withdrawals.
   - **Inefficient Token Transfer**: The `token.transfer` and `token.transferFrom` functions can be gas-intensive, especially for larger token transfers. You should consider batching multiple transfers together or using a more efficient token transfer mechanism, such as the ERC-1155 standard, if applicable.

3. **Best Practices:**
   - **Error Handling**: The contract should provide more detailed error messages to help users understand what went wrong during their interactions. This can be achieved by using custom errors or providing more descriptive error messages in the `require` statements.
   - **Event Emission**: The contract should emit events for important actions, such as deposits, withdrawals, and emergency withdrawals. This allows external applications to monitor and react to the contract's activities.
   - **Pausability**: Consider adding a pausability feature to the contract, which would allow the contract owner to temporarily suspend certain functions in case of an emergency or a security issue.
   - **Upgradability**: If the contract is intended to be used in a production environment, you should consider implementing an upgradable design, such as using the OpenZeppelin Upgrades Plugins, to allow for future updates and bug fixes.
   - **Auditing and Testing**: Before deploying the contract, ensure that it undergoes thorough auditing and testing, including unit tests, integration tests, and security audits, to identify and address any additional issues.

Overall, the provided contract has some security vulnerabilities and gas inefficiencies that should be addressed before deployment. Implementing the recommended best practices can help improve the contract's security, gas efficiency, and maintainability.